In [1]:
import pandas as pd
import numpy as np

import sklearn.linear_model as sklm
import sklearn.model_selection as skms
import sklearn.metrics as metrics
import sklearn.pipeline as skpl
import sklearn.preprocessing as skpp
import sklearn.compose as skcmp
import scipy
import optuna

import os
import gc

In [6]:
df = pd.read_csv("predictions/v2/autogluon_test.csv", index_col="id")
df["class"].replace({0:"e", 1:"p"}).to_csv("autogluon_nan-cat.csv")